# Query Orders  Data as Json  string

### Vamos a reprocesar el archivo que era formato json lo pasamos a txt para poder ver los errores y manejarlo en esta session

In [0]:
SELECT *fROM gizmobox.bronze.v_orders;

### Databricks tiene la capacidad  de extraer columnas de un txt como si fuera un json

Pero esto es para analisis rapido y no es muy utilizado

### 1. EXTRACT TOP LEVEL OBJECTS VALUES   
TOP LEVEL  OBJECT column_name:extract_path

VAMOS SUPONER QUE QUEREMOS EXTRAER el order_id

In [0]:
SELECT 
value:order_id as order_id,
value
FROM gizmobox.bronze.v_orders 

### Ahora vamos a ver como obtener los datos de las matrices o listas anidadas

### EXTRACT ARRAY ELEMENTS
column_name:extraction_path[index>]

In [0]:
SELECT 
value:items[0] as item_1,
value:items[1] as item_2,
value
FROM gizmobox.bronze.v_orders 

In [0]:
SELECT 
value:items[0]["item_id"]
value
FROM gizmobox.bronze.v_orders 

In [0]:
SELECT 
value:items[0].item_id
value
FROM gizmobox.bronze.v_orders 

In [0]:
SELECT 
value:items[0].item_id::INTEGER as item_1_item_id,-- ESTO SIRVE PARA CAMBIAR EL TIPOO DE DATO DE LA COLUMNA
value
FROM gizmobox.bronze.v_orders 

# TRANFOMAMOS EL ARCHIVO ORDERS DATA A UN JSON FORMA GENIAL Y RECOMENDADD

Primeramente vamos a solucionar los problemas de datos y luego lo convertimos aun json

In [0]:
SELECT *FROM gizmobox.bronze.v_orders;

# REGEXP REPLACE : para poder extraer la fecha y luego ponerlos comillas

Lo que hace es extraer y remplazar por un valor o patron

In [0]:
SELECT value,
regexp_replace(value,'"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') as fixed_value
FROM gizmobox.bronze.v_orders;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_orders_fixed
AS
SELECT value,
regexp_replace(value,'"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') as fixed_value
FROM gizmobox.bronze.v_orders;

### SE recomienda corvertirlo a un json y luego trabajarlo e

### AHORA  EL STRING JSON LO VAMOS A CONVERTIR EN  JASON OBJECT

### FROM_JSON

PARA PODER CREAR NUESTRO SCHEMA HAY UNA FUNCION MUY UTILI EN SPARK: SCHEMA_OF_JSON(sTRINGJSON)

In [0]:
SELECT 
fixed_value
FROM tv_orders_fixed

In [0]:
SELECT 
schema_of_json(fixed_value) as shema
FROM tv_orders_fixed
limit 1

In [0]:
SELECT 
from_json(fixed_value,
        'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>' ) as json_value,
        fixed_value
FROM tv_orders_fixed
limit 1

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders_json
AS
SELECT 
from_json(fixed_value,
        'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>' ) as json_value
FROM tv_orders_fixed;
